In [2]:
#第9章/加载编码工具
from transformers import BertTokenizer

token = BertTokenizer.from_pretrained('bert-base-chinese')

token

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [3]:
#第9章/试编码句子
out = token.batch_encode_plus(
    batch_text_or_text_pairs=[('不是一切大树，', '都被风暴折断。'),
                              ('不是一切种子，', '都找不到生根的土壤。')],
    truncation=True,
    padding='max_length',
    max_length=18,
    return_tensors='pt',
    return_length=True,
)

#查看编码输出
for k, v in out.items():
    print(k, v.shape)

#把编码还原为句子
print(token.decode(out['input_ids'][0]))

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


input_ids torch.Size([2, 18])
token_type_ids torch.Size([2, 18])
length torch.Size([2])
attention_mask torch.Size([2, 18])
[CLS] 不 是 一 切 大 树 ， [SEP] 都 被 风 暴 折 断 。 [SEP] [PAD]


In [4]:
#第9章/定义数据集
import torch
from datasets import load_from_disk
import random


class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        dataset = load_from_disk('./data/ChnSentiCorp')[split]

        def f(data):
            return len(data['text']) > 40

        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']

        #切分一句话为前半句和后半句
        sentence1 = text[:20]
        sentence2 = text[20:40]

        #随机整数，取值范围为0和1
        label = random.randint(0, 1)

        #有一半的概率把后半句替换为一句无关的话
        if label == 1:
            j = random.randint(0, len(self.dataset) - 1)
            sentence2 = self.dataset[j]['text'][20:40]

        return sentence1, sentence2, label


dataset = Dataset('train')

sentence1, sentence2, label = dataset[7]

len(dataset), sentence1, sentence2, label

Loading cached processed dataset at data/ChnSentiCorp/train/cache-9d6ffbe8cbc206e6.arrow


(8001, '地理位置佳，在市中心。酒店服务好、早餐品', '大。人品不好，发现1个暗点，都没地哭去。', 1)

In [5]:
#第9章/定义计算设备
import torch

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

device

'cuda'

In [6]:
#第9章/数据整理函数
def collate_fn(data):
    sents = [i[:2] for i in data]
    labels = [i[2] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=45,
                                   return_tensors='pt',
                                   return_length=True,
                                   add_special_tokens=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    #token_type_ids:第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)

    return input_ids, attention_mask, token_type_ids, labels

In [7]:
#第9章/数据整理函数试算
#模拟一批数据
data = [('酒店还是非常的不错，我预定的是套间，服务', '非常好，随叫随到，结帐非常快。', 0),
        ('外观很漂亮,性价比感觉还不错，功能简', '单,适合出差携带。蓝牙摄象头都有了。', 0),
        ('《穆斯林的葬礼》我已闻名很久，只是一直没', '怎能享受4星的服务，连空调都不能用的。', 1)]

#试算
input_ids, attention_mask, token_type_ids, labels = collate_fn(data)

#把编码还原为句子
print(token.decode(input_ids[0]))

input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

[CLS] 酒 店 还 是 非 常 的 不 错 ， 我 预 定 的 是 套 间 ， 服 务 [SEP] 非 常 好 ， 随 叫 随 到 ， 结 帐 非 常 快 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


(torch.Size([3, 45]),
 torch.Size([3, 45]),
 torch.Size([3, 45]),
 tensor([0, 0, 1], device='cuda:0'))

In [7]:
#第9章/数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

len(loader)

1000

In [8]:
#第9章/查看数据样例
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

(torch.Size([8, 45]),
 torch.Size([8, 45]),
 torch.Size([8, 45]),
 tensor([0, 1, 0, 0, 1, 0, 0, 0], device='cuda:0'))

In [9]:
#第9章/加载预训练模型
from transformers import BertModel

pretrained = BertModel.from_pretrained('bert-base-chinese')

#统计参数量
sum(i.numel() for i in pretrained.parameters()) / 10000

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


10226.7648

In [10]:
#第9章/不训练预训练模型,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

In [11]:
#第9章/预训练模型试算
#设定计算设备
pretrained.to(device)

#模型试算
out = pretrained(input_ids=input_ids,
                 attention_mask=attention_mask,
                 token_type_ids=token_type_ids)

out.last_hidden_state.shape

torch.Size([8, 45, 768])

In [34]:
#第9章/定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(out.last_hidden_state[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

#设定计算设备
model.to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

torch.Size([8, 2])

In [35]:
#第9章/训练
from transformers import AdamW
from transformers.optimization import get_scheduler


def train():
    #定义优化器
    optimizer = AdamW(model.parameters(), lr=5e-5)

    #定义loss函数
    criterion = torch.nn.CrossEntropyLoss()

    #定义学习率调节器
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    #模型切换到训练模式
    model.train()

    #按批次遍历训练集中的数据
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader):

        #模型计算
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        #计算loss并使用梯度下降法优化模型参数
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        #输出各项数据的情况，便于观察
        if i % 20 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            print(i, loss.item(), lr, accuracy)


train()

0 0.681793749332428 4.995e-05 0.625
20 0.6200282573699951 4.8950000000000004e-05 0.875
40 0.608764111995697 4.795e-05 0.75
60 0.6029558181762695 4.695e-05 0.75
80 0.49943840503692627 4.5950000000000006e-05 1.0
100 0.5959280729293823 4.495e-05 0.75
120 0.5847670435905457 4.3950000000000004e-05 0.75
140 0.5122112035751343 4.295e-05 1.0
160 0.4979666471481323 4.195e-05 1.0
180 0.5046210289001465 4.095e-05 0.875
200 0.5604161024093628 3.995e-05 0.75
220 0.4611266255378723 3.8950000000000005e-05 1.0
240 0.44622868299484253 3.795e-05 1.0
260 0.41288116574287415 3.6950000000000004e-05 1.0
280 0.5807414650917053 3.595e-05 0.75
300 0.43091052770614624 3.495e-05 1.0
320 0.4966319799423218 3.3950000000000005e-05 0.875
340 0.5417488813400269 3.295e-05 0.875
360 0.44159558415412903 3.1950000000000004e-05 1.0
380 0.4915757179260254 3.095e-05 0.75
400 0.4184477627277374 2.995e-05 1.0
420 0.5038822889328003 2.895e-05 0.875
440 0.46791255474090576 2.7950000000000005e-05 0.875
460 0.46281614899635315 2.

In [36]:
#第9章/测试
def test():
    #定义测试数据集加载器
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    #下游任务模型切换到运行模式
    model.eval()
    correct = 0
    total = 0

    #按批次遍历测试集中的数据
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        #计算5个批次即可，不需要全部遍历
        if i == 5:
            break

        print(i)

        #计算
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        pred = out.argmax(dim=1)

        #统计正确率
        correct += (pred == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()

Loading cached processed dataset at data/ChnSentiCorp/test/cache-0092adaf649d6010.arrow


0
1
2
3
4
0.89375
